**Bureau and Bureau Balance dfs - EDA + Feature Engineering**

# Imports

## Packages

In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import zscore
import warnings
warnings.filterwarnings('ignore')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Data

In [4]:
file_path="../raw_data/"

In [5]:
bureau = pd.read_csv(os.path.join(file_path,'bureau.csv'))

In [6]:
bureau_balance = pd.read_csv(os.path.join(file_path,'bureau_balance.csv'))

In [7]:
names = ['bureau','bureau_balance',]
shape = [bureau.shape[0],bureau_balance.shape[0]]
unique = [bureau.groupby('SK_ID_CURR').count().shape[0],
          bureau_balance.groupby('SK_ID_BUREAU').count().shape[0]]
print(f'bureau has {bureau.shape[0]} rows and {bureau.shape[1]} columns')
print(f'bureau_balance has {bureau_balance.shape[0]} rows and {bureau_balance.shape[1]} columns')
pd.DataFrame(list(zip(names,shape,unique)),
               columns =['df_name', 'rows', 'unique_rows'])

bureau has 1716428 rows and 17 columns
bureau_balance has 27299925 rows and 3 columns


,df_name,rows,unique_rows
0,bureau,1716428,305811
1,bureau_balance,27299925,817395


# Feature Eng

## Running to-do list
* Delete columns used in util, etc after group disc.
* Issue with encoding Credit_Type
* .agg for other columns - ask Christian for code with dict

## Total credit utilisation

In [8]:
# Utilisation calculation - put into function
# Calculate utilisation based on AMT_CREDIT_SUM_LIMIT and AMT_CREDIT_SUM
# drop AMT_CREDIT_SUM, AMT_CREDIT_SUM_DEBT, and AMT_CREDIT_SUM_LIMIT columns
bureau['debt_credit_ratio_limit']=bureau['AMT_CREDIT_SUM_DEBT'] / bureau['AMT_CREDIT_SUM_LIMIT'] 
# bureau[bureau['CREDIT_ACTIVE']=='Active'].head(60)
bureau['debt_credit_ratio_sum']=bureau['AMT_CREDIT_SUM_DEBT'] / bureau['AMT_CREDIT_SUM'] 
#bureau.drop(columns=['AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT','AMT_CREDIT_SUM_LIMIT'])
bureau_drop = ['AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT','AMT_CREDIT_SUM_LIMIT']
print(bureau.shape)
print(len(bureau_drop))

(1716428, 19)
3


In [9]:
# Test above by checking the following with diff credit types
bureau[bureau['CREDIT_TYPE']=='Mortgage']

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,debt_credit_ratio_limit,debt_credit_ratio_sum
12,162297,5714474,Active,currency 1,-277,0,5261.0,NaN,0.0,0,7033500.0,NaN,NaN,0.0,Mortgage,-31,NaN,NaN,NaN
108,419892,5714596,Active,currency 1,-381,0,NaN,NaN,NaN,0,2767500.0,2739622.5,NaN,0.0,Mortgage,-27,NaN,NaN,0.989927
343,450549,5714908,Closed,currency 1,-2250,0,3287.0,-406.0,0.0,0,7794000.0,NaN,NaN,0.0,Mortgage,-406,NaN,NaN,NaN
383,273814,5714968,Closed,currency 1,-186,0,NaN,-118.0,NaN,0,1935000.0,NaN,NaN,0.0,Mortgage,-111,NaN,NaN,NaN
384,273814,5714969,Active,currency 1,-1075,0,4387.0,NaN,NaN,0,5355000.0,NaN,NaN,0.0,Mortgage,-42,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1715600,448157,5043236,Active,currency 1,-1441,0,2197.0,NaN,NaN,0,1800000.0,1076809.5,0.0,0.0,Mortgage,-2,NaN,inf,0.598228
1715800,314708,5051135,Active,currency 1,-1302,0,6003.0,NaN,NaN,0,6300000.0,5978488.5,0.0,0.0,Mortgage,-17,0.0,inf,0.948966
1715877,145500,5054696,Active,currency 1,-2740,0,4565.0,NaN,NaN,0,8262000.0,5500062.0,0.0,0.0,Mortgage,-10,0.0,inf,0.665706
1716173,202163,5049709,Closed,currency 1,-2452,0,3006.0,-44.0,NaN,0,9720000.0,7661722.5,0.0,0.0,Mortgage,-39,NaN,inf,0.788243


## Calculate percent overdue
* AMT_CREDIT_SUM_OVERDUE is 99.8% zeros; recommend dropping
* AMT_CREDIT_MAX_OVERDUE IS 65.5% missing and has valuable info; nans can be filled with zero

In [10]:
# Percent overdue calculation - put into function
# Calculate utilisation based on AMT_CREDIT_SUM_LIMIT and AMT_CREDIT_SUM
# drop AMT_CREDIT_SUM, AMT_CREDIT_SUM_OVERDUE, and AMT_CREDIT_SUM_LIMIT columns
bureau['overdue_percent_limit']=bureau['AMT_CREDIT_MAX_OVERDUE'] / bureau['AMT_CREDIT_SUM_LIMIT'] 
# bureau[bureau['CREDIT_ACTIVE']=='Active'].head(60)
bureau['overdue_percent_sum']=bureau['AMT_CREDIT_MAX_OVERDUE'] / bureau['AMT_CREDIT_SUM'] 
bureau_drop +=['AMT_CREDIT_SUM_OVERDUE','AMT_CREDIT_MAX_OVERDUE']
print(bureau.shape)
print(len(bureau_drop))

(1716428, 21)
5


In [11]:
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,debt_credit_ratio_limit,debt_credit_ratio_sum,overdue_percent_limit,overdue_percent_sum
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,NaN,0.00000,NaN,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,NaN,0.76152,NaN,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,NaN,NaN,NaN,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,NaN,NaN,NaN,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,NaN,NaN,NaN,0.028768


## Encoding Bureau

## Credit_Active

In [12]:
# Encode Credit_Active Column
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)

credit_act = bureau[['CREDIT_ACTIVE']]
credit_act_OHE = ohe.fit_transform(credit_act)
f = ohe.get_feature_names()
print(f)
bureau[f[0]],bureau[f[1]],bureau[f[2]],bureau[f[3]]=credit_act_OHE.T
bureau_drop += ['CREDIT_ACTIVE']
print(bureau.shape)
print(len(bureau_drop))

['x0_Active' 'x0_Bad debt' 'x0_Closed' 'x0_Sold']
(1716428, 25)
6


In [13]:
# Double-check encoder
bureau[bureau['CREDIT_ACTIVE']=='Sold']

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,debt_credit_ratio_limit,debt_credit_ratio_sum,overdue_percent_limit,overdue_percent_sum,x0_Active,x0_Bad debt,x0_Closed,x0_Sold
541,161678,5715165,Sold,currency 1,-1319,0,-1134.0,-75.0,133308.81,0,202500.0,0.0,0.0,0.0,Consumer credit,-71,NaN,NaN,0.0,inf,0.658315,0.0,0.0,0.0,1.0
746,147546,5715420,Sold,currency 1,-1337,0,-239.0,NaN,NaN,0,477000.0,0.0,NaN,0.0,Consumer credit,-280,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,1.0
1077,113548,5715804,Sold,currency 1,-151,0,764.0,NaN,NaN,0,435973.5,NaN,NaN,0.0,Consumer credit,-24,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0
1533,163163,5716347,Sold,currency 1,-2047,0,-1499.0,NaN,NaN,0,153405.0,NaN,NaN,0.0,Credit card,-1957,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0
2493,351664,5717545,Sold,currency 1,-1911,0,-815.0,-134.0,42988.50,0,222750.0,NaN,NaN,0.0,Consumer credit,-22,NaN,NaN,NaN,NaN,0.192990,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1714457,347814,5099924,Sold,currency 1,-2095,0,-1911.0,NaN,NaN,0,23589.0,NaN,NaN,0.0,Credit card,-2057,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0
1715449,102875,5052790,Sold,currency 1,-2327,0,-1960.0,NaN,NaN,0,174222.0,NaN,NaN,0.0,Credit card,-2113,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0
1715602,370974,5043238,Sold,currency 1,-2301,0,-1839.0,NaN,NaN,0,1642500.0,0.0,NaN,0.0,Car loan,-1836,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,1.0
1715907,438184,5054951,Sold,currency 1,-1284,0,-188.0,NaN,NaN,0,675000.0,0.0,NaN,0.0,Consumer credit,-1151,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,1.0


## Credit_Type

In [14]:
credit_type = pd.get_dummies(bureau['CREDIT_TYPE'])
credit_type['SK_ID_BUREAU'] = bureau['SK_ID_BUREAU']
bureau = bureau.merge(credit_type,how='inner', on='SK_ID_BUREAU')

In [15]:
bureau_drop += ['CREDIT_TYPE']
print(bureau.shape)
print(len(bureau_drop))
bureau_cols_to_be_feat = ['CREDIT_CURRENCY', 'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 
                          'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'CNT_CREDIT_PROLONG', 
                           'DAYS_CREDIT_UPDATE', 'AMT_ANNUITY']
len(bureau_cols_to_be_feat)

(1716428, 40)
7


8

In [16]:
bureau_to_be_feat = bureau[['CREDIT_CURRENCY', 'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 
                          'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'CNT_CREDIT_PROLONG', 
                           'DAYS_CREDIT_UPDATE', 'AMT_ANNUITY']]
bureau_to_be_feat.isnull().sum().sort_values(ascending=False)/len(bureau_to_be_feat)

AMT_ANNUITY            0.714735
DAYS_ENDDATE_FACT      0.369170
DAYS_CREDIT_ENDDATE    0.061496
CREDIT_CURRENCY        0.000000
DAYS_CREDIT            0.000000
CREDIT_DAY_OVERDUE     0.000000
CNT_CREDIT_PROLONG     0.000000
DAYS_CREDIT_UPDATE     0.000000
dtype: float64

## drop Bureau columns

In [17]:
bureau_drop

['AMT_CREDIT_SUM',
 'AMT_CREDIT_SUM_DEBT',
 'AMT_CREDIT_SUM_LIMIT',
 'AMT_CREDIT_SUM_OVERDUE',
 'AMT_CREDIT_MAX_OVERDUE',
 'CREDIT_ACTIVE',
 'CREDIT_TYPE']

In [18]:
bureau = bureau.drop(columns=bureau_drop)

## Bureau Balance 

### encode

In [19]:
bureau_balance_status = pd.get_dummies(bureau_balance['STATUS'])
bureau_balance_status['SK_ID_BUREAU'] = bureau_balance['SK_ID_BUREAU']

In [20]:
bureau_balance = pd.concat([bureau_balance,bureau_balance_status.drop('SK_ID_BUREAU',axis=1)],axis=1)

### drop

In [21]:
bureau_balance = bureau_balance.drop(columns=['STATUS'])

### agg

In [22]:
bureau_bal_agg = bureau_balance.groupby('SK_ID_BUREAU').agg(['count','mean','max','min','std','nunique'])

# Bureau and Bureau Balance

## merge and drop

In [23]:
bureau_merged = bureau.merge(bureau_bal_agg,on='SK_ID_BUREAU')
bureau_merged = bureau_merged.drop(columns=['CREDIT_CURRENCY'])
bureau_merged

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,CNT_CREDIT_PROLONG,DAYS_CREDIT_UPDATE,AMT_ANNUITY,debt_credit_ratio_limit,debt_credit_ratio_sum,overdue_percent_limit,overdue_percent_sum,x0_Active,x0_Bad debt,x0_Closed,x0_Sold,Another type of loan,Car loan,Cash loan (non-earmarked),Consumer credit,Credit card,Interbank credit,Loan for business development,Loan for purchase of shares (margin lending),Loan for the purchase of equipment,Loan for working capital replenishment,Microloan,Mobile operator loan,Mortgage,Real estate loan,Unknown type of loan,"(MONTHS_BALANCE, count)","(MONTHS_BALANCE, mean)","(MONTHS_BALANCE, max)","(MONTHS_BALANCE, min)","(MONTHS_BALANCE, std)","(MONTHS_BALANCE, nunique)","(0, count)","(0, mean)","(0, max)","(0, min)","(0, std)","(0, nunique)","(1, count)","(1, mean)","(1, max)","(1, min)","(1, std)","(1, nunique)","(2, count)","(2, mean)","(2, max)","(2, min)","(2, std)","(2, nunique)","(3, count)","(3, mean)","(3, max)","(3, min)","(3, std)","(3, nunique)","(4, count)","(4, mean)","(4, max)","(4, min)","(4, std)","(4, nunique)","(5, count)","(5, mean)","(5, max)","(5, min)","(5, std)","(5, nunique)","(C, count)","(C, mean)","(C, max)","(C, min)","(C, std)","(C, nunique)","(X, count)","(X, mean)","(X, max)","(X, min)","(X, std)","(X, nunique)"
0,380361,5715448,-820,0,31069.0,NaN,0,-183,0.0,0.000000,0.000000,NaN,NaN,1.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,27,-13.0,0,-26,7.937254,27,27,0.296296,1,0,0.465322,2,27,0.0,0,0,0.0,1,27,0.0,0,0,0.0,1,27,0.0,0,0,0.0,1,27,0.0,0,0,0.0,1,27,0.0,0,0,0.0,1,27,0.333333,1,0,0.480384,2,27,0.370370,1,0,0.492103,2
1,380361,5715449,-357,0,1119.0,NaN,0,-130,2691.0,0.000000,0.000000,NaN,NaN,1.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,12,-5.5,0,-11,3.605551,12,12,0.416667,1,0,0.514929,2,12,0.0,0,0,0.0,1,12,0.0,0,0,0.0,1,12,0.0,0,0,0.0,1,12,0.0,0,0,0.0,1,12,0.0,0,0,0.0,1,12,0.500000,1,0,0.522233,2,12,0.083333,1,0,0.288675,2
2,380361,5715451,-917,0,-187.0,-759.0,0,-748,0.0,NaN,0.000000,NaN,NaN,0.0,0.0,1.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,26,-17.5,-5,-30,7.648529,26,26,0.653846,1,0,0.485165,2,26,0.0,0,0,0.0,1,26,0.0,0,0,0.0,1,26,0.0,0,0,0.0,1,26,0.0,0,0,0.0,1,26,0.0,0,0,0.0,1,26,0.192308,1,0,0.401918,2,26,0.153846,1,0,0.367946,2
3,380361,5715452,-993,0,31039.0,-831.0,0,-818,0.0,NaN,0.000000,NaN,NaN,0.0,0.0,1.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,33,-16.0,0,-32,9.669540,33,33,0.242424,1,0,0.435194,2,33,0.0,0,0,0.0,1,33,0.0,0,0,0.0,1,33,0.0,0,0,0.0,1,33,0.0,0,0,0.0,1,33,0.0,0,0,0.0,1,33,0.454545,1,0,0.505650,2,33,0.303030,1,0,0.466694,2
4,380361,5715453,-1146,0,681.0,-780.0,0,-769,0.0,NaN,0.000000,NaN,NaN,0.0,0.0,1.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,38,-18.5,0,-37,11.113055,38,38,0.210526,1,0,0.413155,2,38,0.0,0,0,0.0,1,38,0.0,0,0,0.0,1,38,0.0,0,0,0.0,1,38,0.0,0,0,0.0,1,38,0.0,0,0,0.0,1,38,0.526316,1,0,0.506009,2,38,0.263158,1,0,0.446258,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774349,229054,5053730,-456,0,1372.0,NaN,0,-3,NaN,16.004489,0.941192,0.0,0.0,1.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,15,-7.0,0,-14,4.472136,15,15,0.733333,1,0,0.457738,2,15,0.0,0,0,0.0,1,15,0.0,0,0,0.0,1,15,0.0,0,0,0.0,1,15,0.0,0,0,0.0,1,15,0.0,0,0,0.0,1,15,0.200000,1,0,0.414039,2,15,0.066667,1,0,0.258199,2
774350,254479,5053734,-1571,0,986.0,NaN,0,-2,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,52,-25.5,0,-51,15.154757,52,52,0.134615,1,0,0.344642,2,52,0.0,0,0,0.0,1,52,0.0,0,0,0.0,1,52,0.0,0,0,0.0,1,52,0.0,0,0,0.0,1,52,0.0,0,0,0.0,1,52,0.865385,1,0,0.344642,2,52,0.000000,0,0,0.000000,1
774351,363575,5053740,-1234,0,-503.0,-502.0,0,-461,58554.0,NaN,0.000000,NaN,NaN,0.0,0.0,1.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,41,-20.0,0,-40,11

In [24]:
# agg SK_ID_BUREAU column
bureau_count = bureau_merged[['SK_ID_CURR','SK_ID_BUREAU']]
bureau_count = bureau_count.groupby('SK_ID_CURR').agg(['count'])

# flatten multi-index column names
bureau_count.columns = bureau_count.columns.to_flat_index().str.join('_')

bureau_count

,SK_ID_BUREAU_count
SK_ID_CURR,
100001,7
100002,8
100005,3
100010,2
100013,4
...,...
456247,11
456250,3
456253,4


In [25]:
# merge with bureau_merged
bureau_merged_f = bureau_merged.merge(bureau_count,on='SK_ID_CURR')
bureau_merged_f = bureau_merged_f.drop(columns=['SK_ID_BUREAU'])

## agg
* Merge both dfs on SK_ID_CURR

In [26]:
# aggregate columns
bureau_merged_agg = bureau_merged_f.groupby('SK_ID_CURR').agg(['count','mean','max','min','std','nunique'])



In [27]:
bureau_merged_agg

DAYS_CREDIT                                               CREDIT_DAY_OVERDUE                           DAYS_CREDIT_ENDDATE                                                    DAYS_ENDDATE_FACT                                                  CNT_CREDIT_PROLONG                                     DAYS_CREDIT_UPDATE                                            AMT_ANNUITY                                                            debt_credit_ratio_limit                                 debt_credit_ratio_sum                                            overdue_percent_limit                                 overdue_percent_sum                                                 x0_Active                                       x0_Bad debt                             x0_Closed                                       x0_Sold                             Another type of loan                           Car loan                               Cash loan (non-earmarked)                            \
                 count         mean   max   min          std nunique              count mean max min  std nunique               count         mean      max     min          std nunique             count         mean     max     min         std nunique              count      mean max min       std nunique              count        mean  max   min         std nunique       count           mean       max        min            std nunique                   count mean  max       min std nunique                 count      mean       max  min       std nunique                 count mean  max       min std nunique               count      mean       max       min       std nunique     count      mean  max  min       std nunique       count mean  max  min  std nunique     count      mean  max  min       std nunique   count mean  max  min  std nunique                count mean max min  std nunique    count mean max min      std nunique                     count mean max min  std nunique   
SK_ID_CURR                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
100001               7  -735.000000   -49 -1572   489.942514       7                  7  0.0   0   0  0.0       1                   7    82.428571   1778.0 -1329.0  1032.859277       7                 4  -825.500000  -544.0 -1328.0  369.078582       4                  7  0.000000   0   0  0.000000       1                  7  -93.142857   -6  -155   77.204120       4           7    3545.357143   10822.5      0.000    4800.607529       4                       2  inf  inf       inf NaN       1                     7  0.282518  0.987405  0.0  0.399523       4                     0  NaN  NaN       NaN NaN       0                   0       NaN       NaN       NaN       NaN       0         7  0.428571  1.0  0.0  0.534522       2           7  0.0  0.0  0.0  0.0       1         7  0.571429  1.0  0.0  0.534522       2       7  0.0  0.0  0.0  0.0       1                    7  0.0   0   0  0.0       1        7  0.0   0   0  0.00000       1                         7  0.0   0   0  0.0       1   
100002               8  -87

# TBD: None of the flatten multi-index methods are working

In [37]:
a = bureau_merged_agg
# flatten multi-index column names
a.columns = a.columns.to_flat_index().str.join('_')
a

,DAYS_CREDIT_count,DAYS_CREDIT_mean,DAYS_CREDIT_max,DAYS_CREDIT_min,DAYS_CREDIT_std,DAYS_CREDIT_nunique,CREDIT_DAY_OVERDUE_count,CREDIT_DAY_OVERDUE_mean,CREDIT_DAY_OVERDUE_max,CREDIT_DAY_OVERDUE_min,CREDIT_DAY_OVERDUE_std,CREDIT_DAY_OVERDUE_nunique,DAYS_CREDIT_ENDDATE_count,DAYS_CREDIT_ENDDATE_mean,DAYS_CREDIT_ENDDATE_max,DAYS_CREDIT_ENDDATE_min,DAYS_CREDIT_ENDDATE_std,DAYS_CREDIT_ENDDATE_nunique,DAYS_ENDDATE_FACT_count,DAYS_ENDDATE_FACT_mean,DAYS_ENDDATE_FACT_max,DAYS_ENDDATE_FACT_min,DAYS_ENDDATE_FACT_std,DAYS_ENDDATE_FACT_nunique,CNT_CREDIT_PROLONG_count,CNT_CREDIT_PROLONG_mean,CNT_CREDIT_PROLONG_max,CNT_CREDIT_PROLONG_min,CNT_CREDIT_PROLONG_std,CNT_CREDIT_PROLONG_nunique,DAYS_CREDIT_UPDATE_count,DAYS_CREDIT_UPDATE_mean,DAYS_CREDIT_UPDATE_max,DAYS_CREDIT_UPDATE_min,DAYS_CREDIT_UPDATE_std,DAYS_CREDIT_UPDATE_nunique,AMT_ANNUITY_count,AMT_ANNUITY_mean,AMT_ANNUITY_max,AMT_ANNUITY_min,AMT_ANNUITY_std,AMT_ANNUITY_nunique,debt_credit_ratio_limit_count,debt_credit_ratio_limit_mean,debt_credit_ratio_limit_max,debt_credit_ratio_limit_min,debt_credit_ratio_limit_std,debt_credit_ratio_limit_nunique,debt_credit_ratio_sum_count,debt_credit_ratio_sum_mean,debt_credit_ratio_sum_max,debt_credit_ratio_sum_min,debt_credit_ratio_sum_std,debt_credit_ratio_sum_nunique,overdue_percent_limit_count,overdue_percent_limit_mean,overdue_percent_limit_max,overdue_percent_limit_min,overdue_percent_limit_std,overdue_percent_limit_nunique,overdue_percent_sum_count,overdue_percent_sum_mean,overdue_percent_sum_max,overdue_percent_sum_min,overdue_percent_sum_std,overdue_percent_sum_nunique,x0_Active_count,x0_Active_mean,x0_Active_max,x0_Active_min,x0_Active_std,x0_Active_nunique,x0_Bad debt_count,x0_Bad debt_mean,x0_Bad debt_max,x0_Bad debt_min,x0_Bad debt_std,x0_Bad debt_nunique,x0_Closed_count,x0_Closed_mean,x0_Closed_max,x0_Closed_min,x0_Closed_std,x0_Closed_nunique,x0_Sold_count,x0_Sold_mean,x0_Sold_max,x0_Sold_min,x0_Sold_std,x0_Sold_nunique,Another type of loan_count,Another type of loan_mean,Another type of loan_max,Another type of loan_min,Another type of loan_std,Another type of loan_nunique,Car loan_count,Car loan_mean,Car loan_max,Car loan_min,Car loan_std,Car loan_nunique,Cash loan (non-earmarked)_count,Cash loan (non-earmarked)_mean,Cash loan (non-earmarked)_max,Cash loan (non-earmarked)_min,Cash loan (non-earmarked)_std,Cash loan (non-earmarked)_nunique,Consumer credit_count,Consumer credit_mean,Consumer credit_max,Consumer credit_min,Consumer credit_std,Consumer credit_nunique,Credit card_count,Credit card_mean,Credit card_max,Credit card_min,Credit card_std,Credit card_nunique,Interbank credit_count,Interbank credit_mean,Interbank credit_max,Interbank credit_min,Interbank credit_std,Interbank credit_nunique,Loan for business development_count,Loan for business development_mean,Loan for business development_max,Loan for business development_min,Loan for business development_std,Loan for business development_nunique,Loan for purchase of shares (margin lending)_count,Loan for purchase of shares (margin lending)_mean,Loan for purchase of shares (margin lending)_max,Loan for purchase of shares (margin lending)_min,Loan for purchase of shares (margin lending)_std,Loan for purchase of shares (margin lending)_nunique,Loan for the purchase of equipment_count,Loan for the purchase of equipment_mean,Loan for the purchase of equipment_max,Loan for the purchase of equipment_min,Loan for the purchase of equipment_std,Loan for the purchase of equipment_nunique,Loan for working capital replenishment_count,Loan for working capital replenishment_mean,Loan for working capital replenishment_max,Loan for working capital replenishment_min,Loan for working capital replenishment_std,Loan for working capital replenishment_nunique,Microloan_count,Microloan_mean,Microloan_max,Microloan_min,Microloan_std,Microloan_nunique,Mobile operator loan_count,Mobile operator loan_mean,Mobile operator loan_max,Mobile operator loan_min,Mobile operator loan_std,Mobile operator loan_nu

# Functions

## Get data

In [28]:
df = pd.read_csv(os.path.join(file_path,'bureau.csv')) # replace with Christian's get_data

In [29]:
df_bal = pd.read_csv(os.path.join(file_path,'bureau_balance.csv'))

## Debt to Credit ratio

In [30]:
def debt_credit_ratio(df):
    """Calculate debt to credit ratio. Pass bureau df"""
    bureau = df
    
    bureau['debt_credit_ratio_limit']=bureau['AMT_CREDIT_SUM_DEBT'] / bureau['AMT_CREDIT_SUM_LIMIT'] 
    
    bureau['debt_credit_ratio_sum']=bureau['AMT_CREDIT_SUM_DEBT'] / bureau['AMT_CREDIT_SUM'] 
    
    return bureau

debt_credit_ratio(df)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,debt_credit_ratio_limit,debt_credit_ratio_sum
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,NaN,0.0,Consumer credit,-131,NaN,NaN,0.00000
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.0,NaN,0.0,Credit card,-20,NaN,NaN,0.76152
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN,NaN,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN,NaN,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,0.0,0.0,Microloan,-19,NaN,inf,1.00000
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,0.0,0.0,Consumer credit,-2493,NaN,NaN,0.00000
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,0.0,Consumer credit,-967,NaN,NaN,NaN
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.0,0.0,0.0,Consumer credit,-1508,NaN,NaN,0.00000


## Percent overdue 

In [31]:
def percent_overdue(df):
    """Calculate percent overdue"""
    
    # Call relevant df using function
    bureau = debt_credit_ratio(df)
    
    bureau['overdue_percent_limit']=bureau['AMT_CREDIT_MAX_OVERDUE'] / bureau['AMT_CREDIT_SUM_LIMIT'] 
    
    bureau['overdue_percent_sum']=bureau['AMT_CREDIT_MAX_OVERDUE'] / bureau['AMT_CREDIT_SUM'] 
    
    return bureau

percent_overdue(df)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,debt_credit_ratio_limit,debt_credit_ratio_sum,overdue_percent_limit,overdue_percent_sum
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,NaN,0.0,Consumer credit,-131,NaN,NaN,0.00000,NaN,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.0,NaN,0.0,Credit card,-20,NaN,NaN,0.76152,NaN,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN,NaN,NaN,NaN,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN,NaN,NaN,NaN,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN,NaN,NaN,NaN,0.028768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,0.0,0.0,Microloan,-19,NaN,inf,1.00000,NaN,0.000000
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,0.0,0.0,Consumer credit,-2493,NaN,NaN,0.00000,inf,0.143624
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,0.0,Consumer credit,-967,NaN,NaN,NaN,NaN,NaN
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.0,0.0,0.0,Consumer credit,-1508,NaN,NaN,0.00000,NaN,NaN


## Encode Credit Active

In [32]:
def encode_credit_active(df):
    """Encode Credit_Active Column"""
    bureau = percent_overdue(df)
    
    ohe = OneHotEncoder(sparse=False)

    credit_act = bureau[['CREDIT_ACTIVE']]
    
    credit_act_OHE = ohe.fit_transform(credit_act)
    
    f = ohe.get_feature_names()
    
    bureau[f[0]],bureau[f[1]],bureau[f[2]],bureau[f[3]]=credit_act_OHE.T
    
    return bureau

encode_credit_active(df)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,debt_credit_ratio_limit,debt_credit_ratio_sum,overdue_percent_limit,overdue_percent_sum,x0_Active,x0_Bad debt,x0_Closed,x0_Sold
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,NaN,0.0,Consumer credit,-131,NaN,NaN,0.00000,NaN,NaN,0.0,0.0,1.0,0.0
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.0,NaN,0.0,Credit card,-20,NaN,NaN,0.76152,NaN,NaN,1.0,0.0,0.0,0.0
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN,NaN,NaN,NaN,0.028768,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,0.0,0.0,Microloan,-19,NaN,inf,1.00000,NaN,0.000000,1.0,0.0,0.0,0.0
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,0.0,0.0,Consumer credit,-2493,NaN,NaN,0.00000,inf,0.143624,0.0,0.0,1.0,0.0
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,0.0,Consumer credit,-967,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.0,0.0,0.0,Consumer credit,-1508,NaN,NaN,0.00000,NaN,NaN,0.0,0.0,1.0,0.0


## Encode Credit Type

In [33]:
def encode_credit_type(df):
    """Encode Credit_Type Column"""
    bureau = encode_credit_active(df)
    
    credit_type = pd.get_dummies(bureau['CREDIT_TYPE'])
    credit_type['SK_ID_BUREAU'] = bureau['SK_ID_BUREAU']
    bureau = bureau.merge(credit_type,how='inner', on='SK_ID_BUREAU')
    
    return bureau

encode_credit_type(df)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,debt_credit_ratio_limit,debt_credit_ratio_sum,overdue_percent_limit,overdue_percent_sum,x0_Active,x0_Bad debt,x0_Closed,x0_Sold,Another type of loan,Car loan,Cash loan (non-earmarked),Consumer credit,Credit card,Interbank credit,Loan for business development,Loan for purchase of shares (margin lending),Loan for the purchase of equipment,Loan for working capital replenishment,Microloan,Mobile operator loan,Mortgage,Real estate loan,Unknown type of loan
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,NaN,0.0,Consumer credit,-131,NaN,NaN,0.00000,NaN,NaN,0.0,0.0,1.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.0,NaN,0.0,Credit card,-20,NaN,NaN,0.76152,NaN,NaN,1.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN,NaN,NaN,NaN,0.028768,1.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,0.0,0.0,Microloan,-19,NaN,inf,1.00000,NaN,0.000000,1.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,0.0,0.0,Consumer credit,-2493,NaN,NaN,0.00000,inf,0.143624,0.0,0.0,1.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,0.0,Consumer credit,-967,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.0,0.0,0.0,Consumer credit,-1508,NaN,NaN,0.00000,NaN,NaN,0.0,0.0,1.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


## Get final Bureau df

In [34]:
def get_bureau_final(df):
    """Final bureau df with encoded columns and new features"""
    bureau = encode_credit_type(df)
    
    bureau_drop = ['AMT_CREDIT_SUM','AMT_CREDIT_SUM_DEBT','AMT_CREDIT_SUM_LIMIT',
                   'AMT_CREDIT_SUM_OVERDUE','AMT_CREDIT_MAX_OVERDUE','CREDIT_ACTIVE','CREDIT_TYPE']
    
    bureau = bureau.drop(columns=bureau_drop)
    
    return bureau

get_bureau_final(df)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,CNT_CREDIT_PROLONG,DAYS_CREDIT_UPDATE,AMT_ANNUITY,debt_credit_ratio_limit,debt_credit_ratio_sum,overdue_percent_limit,overdue_percent_sum,x0_Active,x0_Bad debt,x0_Closed,x0_Sold,Another type of loan,Car loan,Cash loan (non-earmarked),Consumer credit,Credit card,Interbank credit,Loan for business development,Loan for purchase of shares (margin lending),Loan for the purchase of equipment,Loan for working capital replenishment,Microloan,Mobile operator loan,Mortgage,Real estate loan,Unknown type of loan
0,215354,5714462,currency 1,-497,0,-153.0,-153.0,0,-131,NaN,NaN,0.00000,NaN,NaN,0.0,0.0,1.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,215354,5714463,currency 1,-208,0,1075.0,NaN,0,-20,NaN,NaN,0.76152,NaN,NaN,1.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,215354,5714464,currency 1,-203,0,528.0,NaN,0,-16,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,215354,5714465,currency 1,-203,0,NaN,NaN,0,-16,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,215354,5714466,currency 1,-629,0,1197.0,NaN,0,-21,NaN,NaN,NaN,NaN,0.028768,1.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,currency 1,-44,0,-30.0,NaN,0,-19,NaN,inf,1.00000,NaN,0.000000,1.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1716424,100044,5057754,currency 1,-2648,0,-2433.0,-2493.0,0,-2493,NaN,NaN,0.00000,inf,0.143624,0.0,0.0,1.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1716425,100044,5057762,currency 1,-1809,0,-1628.0,-970.0,0,-967,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1716426,246829,5057770,currency 1,-1878,0,-1513.0,-1513.0,0,-1508,NaN,NaN,0.00000,NaN,NaN,0.0,0.0,1.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


## Get bureau_balance final

In [35]:
def get_bureau_balance_final(df):
    """Final bureau_balance df"""
    bureau_balance = df
    
    bureau_balance_status = pd.get_dummies(bureau_balance['STATUS'])
    bureau_balance_status['SK_ID_BUREAU'] = bureau_balance['SK_ID_BUREAU']
    bureau_balance = pd.concat([bureau_balance,bureau_balance_status.drop('SK_ID_BUREAU',axis=1)],axis=1)
    
    bureau_balance = bureau_balance.drop(columns=['STATUS'])
    bureau_bal_agg = bureau_balance.groupby('SK_ID_BUREAU').agg(['count','mean','max','min','std','nunique'])
    
    return bureau_bal_agg

get_bureau_balance_final(df_bal)

MONTHS_BALANCE                                      0                                         1                                      2                               3                               4                               5                                    C                                         X                                    
                      count  mean max min        std nunique count      mean max min       std nunique count      mean max min    std nunique count mean max min  std nunique count mean max min  std nunique count mean max min  std nunique count mean max min       std nunique count      mean max min       std nunique count      mean max min       std nunique
SK_ID_BUREAU                                                                                                                                                                                                                                                                                                                                                          
5001709                  97 -48.0   0 -96  28.145456      97    97  0.000000   0   0  0.000000       1    97  0.000000   0   0  0.000       1    97  0.0   0   0  0.0       1    97  0.0   0   0  0.0       1    97  0.0   0   0  0.0       1    97  0.0   0   0  0.000000       1    97  0.886598   1   0  0.318731       2    97  0.113402   1   0  0.318731       2
5001710                  83 -41.0   0 -82  24.103942      83    83  0.060241   1   0  0.239379       2    83  0.000000   0   0  0.000       1    83  0.0   0   0  0.0       1    83  0.0   0   0  0.0       1    83  0.0   0   0  0.0       1    83  0.0   0   0  0.000000       1    83  0.578313   1   0  0.496831       2    83  0.361446   1   0  0.483340       2
5001711                   4  -1.5   0  -3   1.290994       4     4  0.750000   1   0  0.500000       2     4  0.000000   0   0  0.000       1     4  0.0   0   0  0.0       1     4  0.0   0   0  0.0       1     4  0.0   0   0  0.0       1     4  0.0   0   0  0.000000       1     4  0.000000   0   0  0.000000       1     4  0.250000   1   0  0.500000       2
5001712                  19  -9.0   0 -18   5.627314      19    19  0.526316   1   0  0.512989       2    19  0.000000   0   0  0.000       1    19  0.0   0   0  0.0       1    19  0.0   0   0  0.0       1    19  0.0   0   0  0.0       1    19  0.0   0   0  0.000000       1    19  0.473684   1   0  0.512989       2    19  0.000000   0   0  0.000000       1
5001713                  22 -10.5   0 -21   6.493587      22    22  0.000000   0   0  0.000000       1    22  0.000000   0   0  0.000       1    22  0.0   0   0  0.0       1    22  0.0   0   0  0.0       1    22  0.0   0   0  0.0       1    22  0.0   0   0  0.000000       1    22  0.000000   0   0  0.000000       1    22  1.000000   1   1  0.000000       1
...                     ...   ...  ..  ..        ...     ...   ...       ...  ..  ..       ...     ...   ...       ...  ..  ..    ...     ...   ...  ...  ..  ..  ...     ...   ...  ...  ..  ..  ...     ...   ...  ...  ..  ..  ...     ...   ...  ...  ..  ..       ...     ...   ...       ...  ..  ..       ...     ...   ...       ...  ..  ..       ...     ...
6842884                  48 -23.5   0 -47  14.000000      48    48  0.187500   1   0  0.394443       2    48  0.000000   0   0  0.000       1    48  0.0   0   0  0.0       1    48  0.0   0   0  0.0       1    48  0.0   0   0  0.0       1    48  0.0   0   0  0.000000       1    48  0.416667   1   0  0.498224       2    48  0.395833   1   0  0.494204       2
6842885                  24 -11.5   0 -23   7.071068      24    24  0.500000   1   0  0.510754       2    24  0.000000   0   0  0.000       1    24  0.0   0   0  0.0       1    24  0.0   0   0  0.0       1    24  0.0   0   0  0.0       1    24  0.5   1   0  0.510754       2    24  0.000000   0   0  0.000000       1    24  0.000000   0   0  0.000000       1
6842886                  33 -16.0   0 -32   9.669540      33    

## Merge,agg bureau and bureau_balance 

In [36]:
def get_final_bureau_merged(df1,df2):
    
    bureau = get_bureau_final(df1)
    bureau_bal_agg = get_bureau_balance_final(df2)
    
    bureau_merged = bureau.merge(bureau_bal_agg,on='SK_ID_BUREAU')
    bureau_merged = bureau_merged.drop(columns=['CREDIT_CURRENCY'])
    
    # agg SK_ID_BUREAU column
    bureau_count = bureau_merged[['SK_ID_CURR','SK_ID_BUREAU']]
    bureau_count = bureau_count.groupby('SK_ID_CURR').agg(['count'])

    # flatten multi-index column names
    bureau_count.columns = bureau_count.columns.to_flat_index().str.join('_')
    
    # merge into df
    bureau_merged_f = bureau_merged.merge(bureau_count,on='SK_ID_CURR')
    bureau_merged_f = bureau_merged_f.drop(columns=['SK_ID_BUREAU'])
    
    # agg df
    bureau_merged_agg = bureau_merged_f.groupby('SK_ID_CURR').agg(['count','mean','max','min','std','nunique'])
    
    # return final df of merged bureau and bureau_balance with new and encoded features
    return bureau_merged_agg

get_final_bureau_merged(df,df_bal)

DAYS_CREDIT                                               CREDIT_DAY_OVERDUE                           DAYS_CREDIT_ENDDATE                                                    DAYS_ENDDATE_FACT                                                  CNT_CREDIT_PROLONG                                     DAYS_CREDIT_UPDATE                                            AMT_ANNUITY                                                            debt_credit_ratio_limit                                 debt_credit_ratio_sum                                            overdue_percent_limit                                 overdue_percent_sum                                                 x0_Active                                       x0_Bad debt                             x0_Closed                                       x0_Sold                             Another type of loan                           Car loan                               Cash loan (non-earmarked)                            \
                 count         mean   max   min          std nunique              count mean max min  std nunique               count         mean      max     min          std nunique             count         mean     max     min         std nunique              count      mean max min       std nunique              count        mean  max   min         std nunique       count           mean       max        min            std nunique                   count mean  max       min std nunique                 count      mean       max  min       std nunique                 count mean  max       min std nunique               count      mean       max       min       std nunique     count      mean  max  min       std nunique       count mean  max  min  std nunique     count      mean  max  min       std nunique   count mean  max  min  std nunique                count mean max min  std nunique    count mean max min      std nunique                     count mean max min  std nunique   
SK_ID_CURR                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
100001               7  -735.000000   -49 -1572   489.942514       7                  7  0.0   0   0  0.0       1                   7    82.428571   1778.0 -1329.0  1032.859277       7                 4  -825.500000  -544.0 -1328.0  369.078582       4                  7  0.000000   0   0  0.000000       1                  7  -93.142857   -6  -155   77.204120       4           7    3545.357143   10822.5      0.000    4800.607529       4                       2  inf  inf       inf NaN       1                     7  0.282518  0.987405  0.0  0.399523       4                     0  NaN  NaN       NaN NaN       0                   0       NaN       NaN       NaN       NaN       0         7  0.428571  1.0  0.0  0.534522       2           7  0.0  0.0  0.0  0.0       1         7  0.571429  1.0  0.0  0.534522       2       7  0.0  0.0  0.0  0.0       1                    7  0.0   0   0  0.0       1        7  0.0   0   0  0.00000       1                         7  0.0   0   0  0.0       1   
100002               8  -87